Notebook to run:

```bash
python train_and_eval/segmentation_training_transf.py --config configs/PASTIS24/TSViT-S_fold1.yaml --device 0
```


In [ ]:
import os
import sys


In [ ]:
if "notebooks" in os.getcwd():
    os.chdir(os.path.join("..", "src"))

os.getcwd()


In [ ]:
%env PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python


In [ ]:
from train_and_eval import segmentation_training_transf


In [ ]:
config_file: str = "../configs/PASTIS24/TSViT-S_fold1.yaml"
device_ids: list = [0]

segmentation_training_transf.run(config_file=config_file, device_ids=device_ids)


---

In [2]:

import os
from dotenv import load_dotenv
from azure.ai.ml import command, Input, Output, UserIdentityConfiguration, MLClient
from azure.ai.ml.entities import Environment, Data

# Handle to the workspace
from azure.ai.ml.constants import AssetTypes, InputOutputModes

# Authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential


In [3]:
try:
    load_dotenv()
    subscription_id = os.environ.get("SUBSCRIPTION_ID")
    resource_group_name = os.environ.get("RESOURCE_GROUP")
    workspace_name = os.environ.get("WORKSPACE_NAME")
except Exception as e:
    print(e)
    print(
        "No .env file found. Created one with environment variables SUBSCRIPTION_ID, RESOURCE_GROUP, WORKSPACE_NAME"
    )


In [4]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()


In [5]:
print(f"Using AzureML Workspace `{workspace_name}` within the RG `{resource_group_name}` and Subscription `{subscription_id}`")
ml_client = MLClient(credential, subscription_id, resource_group_name, workspace_name)


Using AzureML Workspace `bioversml` within the RG `semantic_segmentation` and Subscription `8b829ed0-f80d-4b85-8636-a7967eaf7aed`


In [6]:
print("\nAvailable Environments:")
for env in ml_client.environments.list():
    print(f" - {env.name}:{env.latest_version}")



Available Environments:
 - deepsatmodels_env:18
 - python-sdk-v2:20
 - CliV2AnonymousEnvironment:0
 - naturevers_inference_env:2
 - naturevers_dataset_env:1
 - naturevers_azureml:0
 - fallowland:0
 - tritonNcdEnv:0
 - AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu:10
 - AzureML-TensorFlow2.4-Cuda11-OpenMpi4.1.0-py36:1
 - AzureML-Scikit-learn0.24-Cuda11-OpenMpi4.1.0-py36:1
 - AzureML-Pytorch1.7-Cuda11-OpenMpi4.1.0-py36:1


In [7]:
print("Latest Jobs:")
for index, job in enumerate(ml_client.jobs.list()):
    print(f" - '{job.display_name}'")
    if index > 5:
        break


Latest Jobs:
 - 'DeepSatModels Segmentation Training'
 - 'DeepSatModels Segmentation Training'
 - 'DeepSatModels Segmentation Training'
 - 'DeepSatModels Segmentation Training'
 - 'boring_window_6pxm225t'
 - 'DeepSatModels Segmentation Training'
 - 'frosty_cat_9lm2ywzd'


In [8]:
env_name = "naturevers_inference_env"
env_version = "2"

env = ml_client.environments.get(
        env_name, version=str(env_version)
    )


In [9]:
datastore = ml_client.datastores.get("covercropatlas")
print(datastore.name, datastore.type, datastore.account_name, datastore.container_name)

datastore_folder = Input(
    path=datastore.id,
    type=AssetTypes.URI_FOLDER,
    mode=InputOutputModes.MOUNT,
)
display(datastore_folder)


covercropatlas DatastoreType.AZURE_BLOB semsegstorage biovers-covercrop-atlas


{'type': 'uri_folder', 'path': '/subscriptions/8b829ed0-f80d-4b85-8636-a7967eaf7aed/resourceGroups/semantic_segmentation/providers/Microsoft.MachineLearningServices/workspaces/bioversml/datastores/covercropatlas', 'mode': 'mount'}

In [10]:
job_args = {
    "inputs": dict(
        datastore_folder=datastore_folder
    ),
    "outputs": dict(),
    "code": "../src",  # location of source code
    "command": """python aml_job/train_segmentation_model.py""",
    "environment": env,
    "compute": "FALLOWV100WESTLOW",
    "experiment_name": "deepsatmodels_segmentation_training",
    "display_name": "DeepSatModels Segmentation Training",
    "description": "Training segmentation model using DeepSatModels repository",
    "identity": UserIdentityConfiguration(),
}
job = command(**job_args)


In [12]:
# ml_client.create_or_update(job)

print("DONE")


DONE


In [16]:
help('modules')



Please wait a moment while I gather a list of all available modules...

Fail to import hypothesis in common_utils, tests are not derandomized
Cython              caffe2              jupyterlab_server   requests_oauthlib
IPython             calendar            jupyterlab_widgets  resource
OpenSSL             certifi             jwt                 rlcompleter
PIL                 cffi                keras               rpds
PyQt5               cgi                 keras_preprocessing rsa
__future__          cgitb               keyword             runpy
_abc                chainer             kiwisolver          sched
_argon2_cffi_bindings chainercv           knack               scipy
_ast                chainermn           lib2to3             secrets
_asyncio            chainerx            libfuturize         secretstorage
_bisect             chardet             libpasteurize       select
_blake2             charset_normalizer  linecache           selectors
_bootlocale         chunk     

/anaconda/envs/deepsatmodels/lib/python3.8/pkgutil.py:92: MatplotlibDeprecationWarning: 
The matplotlib.compat module was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  __import__(info.name)
/anaconda/envs/deepsatmodels/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
/anaconda/envs/deepsatmodels/lib/python3.8/pkgutil.py:92: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  __import__(info.name)
/anaconda/envs/deepsatmodels/lib/python3.8/site-packages/skimage/viewer/utils/__init__.py:1: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.


In [19]:
!conda list | grep timm


In [17]:
import timm


In [20]:
timm.__version__

'0.4.12'

In [13]:

path: str = os.path.join("..", "datasets.yaml")

datasets_yaml = Input(
    path=path,
    type=AssetTypes.URI_FILE,
    mode=InputOutputModes.MOUNT,
)

datasets_yaml


{'type': 'uri_file', 'path': '../datasets.yaml', 'mode': 'mount'}